# Using ReliableTool to Generate Sub-Questions (Synchronous Version)
This notebook demonstrates how to use the `ReliableTool` synchronously.

In [ ]:
import logging
import os
from typing import Any, Optional

from autogen import LLMConfig, config_list_from_json
from autogen.tools.experimental.google_search import GoogleSearchTool
from autogen.tools.experimental.reliable import ReliableTool

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s: %(message)s")
logger = logging.getLogger(__name__)

print("Successfully imported components from autogen.")


## 1. Define the Google Search Tool


In [ ]:
search_api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

gs_tool: GoogleSearchTool = GoogleSearchTool(
    search_api_key=search_api_key,
    search_engine_id=search_engine_id,
)
logger.info("GoogleSearchTool instance created successfully.")

## 2. Configure LLM and ReliableTool

In [ ]:
llm_config: Optional[LLMConfig] = None
config_list = config_list_from_json(
    "OAI_CONFIG_LIST",
)
llm_config = LLMConfig(
    config_list=config_list,
    temperature=0.7,
)

system_message_addition_for_tool_calling = """You are an assistant that prepares search queries for the Google Search tool.
Your goal is to take a user request (the 'task') and formulate the most effective search query string for the tool to execute.
The internal tool to call is named 'execute_google_search'.
You MUST provide the search query as the 'query' argument to this tool.
You MUST also provide a 'hypothesis' argument summarizing the type of information or answer expected from the search results (e.g., "a list of recent news articles", "a definition", "the official website").

Example:
User Task: "Find recent news about the AutoGen library."
Your Tool Call should be: execute_google_search(query="AutoGen library recent news", hypothesis="A list of recent news articles or blog posts about the AutoGen library.")

Example:
User Task: "What is the capital of France?"
Your Tool Call should be: execute_google_search(query="capital of France", hypothesis="The name of the capital city of France.")

You MUST invoke the 'execute_google_search' tool exactly once per response using a tool call format. Do not add conversational text.
"""

system_message_addition_for_result_validation = (
    "Make sure all search results are specifically from the link github.com."
)

reliable_search_tool: ReliableTool = ReliableTool(
    name="ReliableGoogleSearch",
    func_or_tool=gs_tool,  # Pass the initialized GoogleSearchTool
    description="Reliably performs a Google search based on a task description and validates the relevance of the results.",
    runner_llm_config=llm_config,
    validator_llm_config=llm_config,
    system_message_addition_for_tool_calling=system_message_addition_for_tool_calling,
    system_message_addition_for_result_validation=system_message_addition_for_result_validation,
    max_tool_invocations=4,
)

## 3. Get User Input and Run the Tool Synchronously

In [ ]:
search_task = "What is the latest news about the AutoGen project?"

logger.info(f"\nAttempting to reliably perform search task: '{search_task}'")
print("-" * 40)

result: Optional[Any] = reliable_search_tool.run(
    task=search_task, validation_prompt_addition="Ensure every link comes from github."
)

print("-" * 40)
logger.info("\n✅ Successfully performed search task:")
print(f"   Result Type: {type(result)}")
print(f"   Result: {result}")